In [1]:
# Dependencies
import requests
from pprint import pprint
import pandas as pd

pd.options.mode.chained_assignment = None

In [6]:
# get the closing price of a stock on a certain date.
# by default, API returns a dozen or so fields,
# but the "serietype=line" option changes to return ony closing price.
# this speeds up a fairly slow API
def get_price(price_url, price_date):
    date_url = price_url\
        + "?from=" + price_date\
        + "&to=" + price_date\
        + "&serietype=line"
    date_json = requests.get(date_url).json()
    
    # some stocks first listed mid-year and have no closing 
    # price on 2018-12-31, so need a try
    try:
        prices_list = date_json["historical"]
        date_close = prices_list[0]["close"]
    except:
        date_close = None
    
    return date_close

In [7]:
# Pull data from cleaned CSV
nyse_csv = "Data_after_cleaning.csv"
nyse_df = pd.read_csv(init_csv, index_col=0)
nyse_df = init_df.rename(columns={"LastSale": "Last_Sale"})

In [8]:
# nyse_df = init_df.dropna()

In [9]:
base_url = "https://financialmodelingprep.com/api/v3/historical-price-full/"
start_date = "2018-12-31"

ytd_list = []

for index, row in nyse_df.iterrows():
    symbol_url = base_url + row["Symbol"]
    start_year_price = get_price(symbol_url, start_date)
    ytd_list.append(start_year_price)
    
last_sale_loc = nyse_df.columns.get_loc("Last_Sale")
nyse_df.insert(last_sale_loc + 1, "Start_of_Year_Sale", ytd_list)

nyse_df.head()

,Symbol,Name,Last_Sale,Start_of_Year_Sale,Sector
0,DDD,3D Systems Corporation,8.24,10.1700,Technology
1,MMM,3M Company,164.71,187.6263,Health Care
2,WBAI,500.com Limited,9.88,7.5800,Consumer Services
3,WUBA,58.com Inc.,51.33,54.2100,Technology
4,EGHT,8x8 Inc,23.32,18.0400,Technology


In [10]:
# find percent change of each stock symbol and place it in a series
percent_series = ((nyse_df["Last_Sale"] - nyse_df["Start_of_Year_Sale"]) / nyse_df["Last_Sale"]) * 100

# place percentage change series into our dataframe
year_loc = nyse_df.columns.get_loc("Start_of_Year_Sale")
nyse_df.insert(year_loc + 1, "YTD_Change_Percent", percent_series)
nyse_df.head()

,Symbol,Name,Last_Sale,Start_of_Year_Sale,YTD_Change_Percent,Sector
0,DDD,3D Systems Corporation,8.24,10.1700,-23.422330,Technology
1,MMM,3M Company,164.71,187.6263,-13.913120,Health Care
2,WBAI,500.com Limited,9.88,7.5800,23.279352,Consumer Services
3,WUBA,58.com Inc.,51.33,54.2100,-5.610754,Technology
4,EGHT,8x8 Inc,23.32,18.0400,22.641509,Technology


In [11]:
# output results of API to a csv,
# so other notebooks can access data without calling API

out_csv = "NYSE_Sector_Performance.csv"

# drop rows that had no price at start of year
nyse_df = nyse_df.dropna()

# make Symbol our index and re-sort
nyse_df = nyse_df.set_index("Symbol")
nyse_df = nyse_df.sort_index()
nyse_df.to_csv(out_csv)